In [ ]:
midi_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/ProcessedData/midi_from_json_songs"
musecoco_line_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/DataCrawling/ProcessedData/musecoco_lines"

In [ ]:
midi_lib_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/midi_lib"

In [ ]:
import os
import sys
sys.path.append(midi_lib_path)

from converter.musecoco_line_to_musecoco_line_string import musecoco_line_to_musecoco_line_string_converter
from converter.midi_file_object_to_midi_dto import midi_file_object_to_midi_dto_converter
from converter.midi_dto_to_musecoco_line import midi_dto_to_musecoco_line_converter
from musecoco_original_libs.vocab_manager import VocabManager

import midi_file_utils as mfu

In [ ]:
# List all .mid files in the directory
midi_files = [f for f in os.listdir(midi_path) if f.endswith('.mid')]

n_midi_file = len(midi_files)

print(n_midi_file)

In [ ]:
musecoco_lines = []

current_idx = 0
for midi_file in midi_files:
    print(f"Processing {current_idx}/{n_midi_file}")
    
    current_idx += 1
    
    midi_obj = mfu.read_midi_file(os.path.join(midi_path, midi_file))
    
    midi_dto = midi_file_object_to_midi_dto_converter(midi_obj)
    
    musecoco_line = midi_dto_to_musecoco_line_converter(
        midi_dto,
        VocabManager(
            pos_resolution=1
            # max_ts_denominator_power=vocab_config.max_ts_denominator_power,
            # max_notes_per_bar=vocab_config.max_notes_per_bar,
            # tempo_quant=vocab_config.tempo_quant,
            # min_tempo=vocab_config.min_tempo,
            # max_tempo=vocab_config.max_tempo,
            # velocity_quant=vocab_config.velocity_quant,
            # max_duration=vocab_config.max_duration,
            # max_bar_num=vocab_config.max_bar_num
        )
    )
    
    musecoco_lines.append(musecoco_line)
    print()

In [ ]:
current_idx = 0

file_path = f"{musecoco_line_path}/json_song_musecoco_lines.txt"

min_line_length = 100

# Write each element in musecoco_lines as a line in a file
with open(file_path, 'w') as f:
    for musecoco_line in musecoco_lines:
        print(f"Writing {current_idx}/{n_midi_file}")

        current_idx += 1

        musecoco_line = musecoco_line_to_musecoco_line_string_converter(musecoco_line)

        if len(musecoco_line) < min_line_length:
            print(f"Line is too short, skipping: {musecoco_line}")
            continue
        else:
            line_ending = '\n' if current_idx < n_midi_file else ''

            f.write(f"{musecoco_line}{line_ending}")

            print()

print()
print("-----> DONE")